<a href="https://colab.research.google.com/github/srishti-git1110/Simple_NN_in_PyTorch/blob/main/FFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is just to get a hands on demo of pytorch for ones who know what neural networks are & their working in theory, and want to implement their theoretical knowledge in pytorch.
You can run this nb on a gpu as well as cpu.

We construct a very short network here working on MNIST data.

In [2]:
import torch.nn as nn #ffnn, cnn, rnn etc
import torch.optim as optim #optimisers like sgd etc
import torch.nn.functional as f #activ funcs
from torch.utils.data import DataLoader #minibatch train test etc
import torchvision.datasets as datasets #std datasets
import torchvision.transforms as transforms #transformations to perform on dataset


In [1]:
import torch

In [3]:


class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(in_features=input_size, out_features=50)
    self.fc2 = nn.Linear(in_features=50, out_features=num_classes)

  def forward(self, x): #x is the data examples*features
    x = f.relu(self.fc1(x)) 
    x = self.fc2(x)
    return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784
num_classes = 10
learning_rate = .001
batch_size = 64
epochs = 1

#load data
train_set = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)

test_set = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

#initialize
model = NN(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#train
for epoch in range(epochs):
  for batch_num, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    data = data.reshape(data.shape[0], -1)
    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("checking accuracy on train data")
  else:
    print("checking accuracy on test data")
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x) #64*10
      _, predictions = scores.max(1) #probs, indices
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
    print(f'Accuracy {float(num_correct)/float(num_samples)*100:.2f}')
  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

checking accuracy on train data
Accuracy 93.06
checking accuracy on test data
Accuracy 93.14
